# Оптимизация нейросети
Используем регуляризацию, отсев, пакетную нормализацию и инициализацию весов, чтобы улучшить точность работы нейросети.

Используем архитектуру LeNet.
![Отсев](dropout.png)

### Подключение библиотек

In [1]:
import numpy as np
import keras as K
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Загрузка набора MNIST

In [2]:
(X_train, y_train), (X_test, y_test) = K.datasets.mnist.load_data()
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
y_train = K.utils.to_categorical(y_train, 10)
y_test = K.utils.to_categorical(y_test, 10)

### Оптимизированная модель нейронной сети
Свертка 6@5x5 -> Нормализация -> Подвыборка 2x2 -> Нормализация -> Свертка 16@5x5 -> Нормализация -> Подвыборка 2x2 -> Нормализация -> Полносвязный слой 120 -> Отсев -> Полносвязный слой 84 -> Отсев -> Выходной слой 10

* Регуляризация - ограничение весов нейросети по ElasticNet: L1 (сумма модулей) и L2 (сумма квадратов)
* Отсев - случайное отключение узлов на очередном шаге обучения
* Пакетная нормализация - линейное преобразование данных между слоями
* Инициализация весов - начальные, нормально распределенные, значения весов нейросети

In [3]:
model = K.Sequential()
model.add(K.layers.Conv2D(filters=6, kernel_size=(5,5), activation="relu",
                         input_shape=(28, 28, 1), kernel_initializer="he_normal"))
model.add(K.layers.BatchNormalization())
model.add(K.layers.AveragePooling2D())
model.add(K.layers.Conv2D(filters=16, kernel_size=(5,5), activation="relu", kernel_initializer="he_normal"))
model.add(K.layers.BatchNormalization())
model.add(K.layers.AveragePooling2D())
model.add(K.layers.Flatten())
model.add(K.layers.Dense(units=120, activation="relu",
                         kernel_regularizer=K.regularizers.l1_l2(l1=1e-4, l2=1e-5),
                         kernel_initializer="he_normal"))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dropout(0.2))
model.add(K.layers.Dense(units=84, activation="relu",
                         kernel_regularizer=K.regularizers.l1_l2(l1=1e-4, l2=1e-5),
                         kernel_initializer="he_normal"))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dropout(0.2))
model.add(K.layers.Dense(units=10, activation="softmax"))
model.compile(optimizer="SGD", loss=K.losses.categorical_crossentropy)

### Обучение нейросети

Число пересчетов весов при обратном распространении ошибки = число итераций * число эпох

batch_size ~= число классов * 10, но не больше оперативной памяти

In [4]:
model.fit(X_train, y_train, batch_size=128, epochs=50)

Epoch 1/50
60000/60000 [==============================] - 18s 298us/step - loss: 1.0137
Epoch 2/50
60000/60000 [==============================] - 17s 289us/step - loss: 0.5947
Epoch 3/50
60000/60000 [==============================] - 16s 273us/step - loss: 0.5285
Epoch 4/50
60000/60000 [==============================] - 15s 252us/step - loss: 0.4935
Epoch 5/50
60000/60000 [==============================] - 17s 287us/step - loss: 0.4710
Epoch 6/50
60000/60000 [==============================] - 21s 342us/step - loss: 0.4557
Epoch 7/50
60000/60000 [==============================] - 18s 307us/step - loss: 0.4452
Epoch 8/50
60000/60000 [==============================] - 18s 296us/step - loss: 0.4338
Epoch 9/50
60000/60000 [==============================] - 16s 269us/step - loss: 0.4221
Epoch 10/50
60000/60000 [==============================] - 16s 265us/step - loss: 0.4154
Epoch 11/50
60000/60000 [==============================] - 16s 270us/step - loss: 0.4097
Epoch 12/50
60000/60000 [=====

### Предсказание значений и оценка

In [5]:
r = np.argmax(model.predict(X_test), axis=1)
print (classification_report(r, np.argmax(y_test, axis=1), digits=4))

              precision    recall  f1-score   support

           0     0.9959    0.9929    0.9944       983
           1     0.9991    0.9939    0.9965      1141
           2     0.9893    0.9903    0.9898      1031
           3     0.9901    0.9940    0.9921      1006
           4     0.9969    0.9929    0.9949       986
           5     0.9865    0.9899    0.9882       889
           6     0.9927    0.9906    0.9917       960
           7     0.9922    0.9827    0.9874      1038
           8     0.9887    0.9969    0.9928       966
           9     0.9832    0.9920    0.9876      1000

    accuracy                         0.9916     10000
   macro avg     0.9915    0.9916    0.9915     10000
weighted avg     0.9916    0.9916    0.9916     10000

